### OSMGetPOI.jl Tutorial
Full documentation: https://github.com/mkloe/OSMgetPOI

In [1]:
using OSMgetPOI

### 1. Download .osm data file
- Downloading from Geofabrik
    1. Go to https://download.bbbike.org/osm/bbbike/ and select a city.
    2. Copy URL of *OSM XML gzip'd* file (exemplary URL: https://download.bbbike.org/osm/bbbike/Corsica/Corsica.osm.gz)
- Downloading from BBBike
    1. Go to https://download.geofabrik.de/index.html and select a city or region.
    2. Get URL of *.osm.bz2* file (examplary URL: https://download.geofabrik.de/asia/maldives-latest.osm.bz2)


Downloading file from BBBike

In [2]:
download_bbbike_file("https://download.bbbike.org/osm/bbbike/UlanBator/UlanBator.osm.gz")

File downloaded. Unzipping file...
File saved at 

datasets/file.osm

Downloading file from Geofabrik

In [7]:
download_geofabrik_file("https://download.geofabrik.de/asia/bhutan-latest.osm.bz2")

File downloaded. Unzipping file...
File saved at 

datasets/file.osm

### 2. Generate a dataframe 
Generate a dataframe with the POI types defined in the parameters, based on `src/POITypes`. More information about the supported POI types by this API is provided in the documentation.

In [2]:
schools = get_poi_df("datasets/file.osm", 1.0, ["addr:housenumber", "addr:street", "addr:postcode", "addr:country"], OSMgetPOI.POITypes.education_school)

OSM filter process time: 0.422314083

Row,poi_type,lat,lon,addr:street,addr:housenumber,addr:postcode,addr:country
,Any,Any,Any,Any,Any,Any,Any
1,education_school,47.9209,106.902,missing,missing,missing,missing
2,education_school,47.9166,106.908,missing,missing,missing,missing
3,education_school,47.9284,106.925,missing,missing,missing,missing
4,education_school,47.9179,106.903,missing,missing,missing,missing
5,education_school,47.9294,106.92,missing,missing,missing,missing
6,education_school,47.9136,106.896,Сөүлийн гудамж,missing,missing,missing
7,education_school,47.924,106.946,missing,missing,missing,missing
8,education_school,47.9209,106.927,missing,missing,missing,missing
9,education_school,47.9271,106.92,missing,missing,missing,missing


### 3. Save the results to a .csv file.

In [10]:
using CSV
CSV.write("output_csv/schools.csv", singapore_school_df)

"output_csv/schools.csv"

### 4. Visualising POIs on the map

### Remarks
This research was funded in whole or in part by [National Science Centre,  Poland][2021/41/B/HS4/03349]. For the software’s  documentation for the purpose of Open Access, the author has applied a CC-BY public copyright licence to any Author Accepted Manuscript (AAM) version arising from this submission.